In [1]:
import os
import csv
import time
import pylrc
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta

In [2]:
# 搜尋歌詞
# https://github.com/cjlh/spotify-lyrics/blob/master/spotify-lyrics.py
def print_lyrics(artist,title):
    pageurl = "https://makeitpersonal.co/lyrics?artist=" + artist + "&title=" + title
    lyrics = requests.get(pageurl).text.strip()

    # 如果找不到歌詞的方案
    try:
        if lyrics == "Sorry, We don't have lyrics for this song yet.":
            wiki_url = "https://lyrics.fandom.com/wiki/"
            title = title.replace(" ", "_")
            artist = artist.replace(" ", "_")
            url = wiki_url + f"{artist}:{title}"

            r = requests.get(url)
            soup = BeautifulSoup(r.text, 'html.parser')

            lyric_box = str(soup.find("div", {"class": "lyricbox"}))
            lyrics = lyric_box.replace("<br/>", "\n")
            lyrics = lyrics.replace('<div class="lyricbox">', '')
            lyrics = lyrics.replace('<div class="lyricsbreak">', '')
            lyrics = lyrics.replace('</div>', '')
    except:
        print('None Lyrics')
        check = 1
        return check
        pass
    return lyrics

In [3]:
def save2txt(artist,title,lyrics):
    filename =  'lyrics/' + artist + '_' + title + '.txt'
    f = open(filename,'w',encoding="utf-8")
    f.write(lyrics)
    return filename

In [4]:
# 遍歷資料夾(D:/TuneFab Spotify Music Converter/songs)，查看需要下載的歌詞
def through_path():
    yourPath = r'D:/TuneFab Spotify Music Converter/songs' # mp3資料夾位址
    allFileList = os.listdir(yourPath)
    dir_artist = [] # 儲存歌手的list
    dir_artist2 = [] # 儲存歌手的list(寫入dataframe用)
    dir_artist3 = [] # 儲存歌手原始名稱的list(處理資料夾名稱有多位歌手的情況)
    dir_title = [] # 儲存歌名的list

    for file in allFileList: # 遍歷songs資料夾
        if os.path.isdir(os.path.join(yourPath,file)):
            print("dir: " + file) # 列出songs資料夾底下有那些歌手
            dir_artist.append(file)

            artistPath = yourPath + '/' + file # 形式：D:/TuneFab Spotify Music Converter/songs/{歌手名稱}
            #print(artistPath)
            artistFileList = os.listdir(artistPath)
            #print(artistFileList)

            for file2 in artistFileList: # 遍歷每位歌首的資料夾
                songPath = artistPath + '/' + file2
                if os.path.isfile(songPath): # 形式：D:/TuneFab Spotify Music Converter/songs/{歌手名稱}/{歌曲名稱}
                    #print("dir2: " + file2)
                    file2 = file2.replace('.mp3','').replace("’","'") #取代附檔名
                    dir_title.append(file2)
                    
                    name = songPath.split('/')[-2] # 取songPath位址倒數第二個值
                    name_list = name.split(',') 
                    dir_artist2.append(name_list[0]) # 針對資料夾有多位歌手處理，取第一個為主要演唱者
                    
                    # 原始歌手資料夾名稱
                    originalname = artistPath.split('/')[-1]
                    dir_artist3.append(originalname)

        #print('-'*80)

    # 輸出datafrme(df_songlist)和xlsx(df_songlist.xlsx)
    # 有重複歌曲就刪除，歌曲名稱結尾是_0
    string = '_0|_1|_2'
    df_songlist = pd.DataFrame(list(zip(dir_artist3,dir_artist2,dir_title)),columns=['Original','Artist','Title'])
    df_songlist_select = df_songlist.loc[~df_songlist['Title'].str.contains(string)].reset_index(drop=True)
    df_songlist_select.to_excel('table/df_songlist_select.xlsx',index=0)
    return df_songlist_select

In [5]:
df_songlist_select = through_path()
df_songlist_select

dir: 5 Seconds of Summer
dir: Adele


,Original,Artist,Title
0,5 Seconds of Summer,5 Seconds of Summer,18
1,5 Seconds of Summer,5 Seconds of Summer,Airplanes
2,5 Seconds of Summer,5 Seconds of Summer,Amnesia
3,5 Seconds of Summer,5 Seconds of Summer,Babylon
4,5 Seconds of Summer,5 Seconds of Summer,Beside You
...,...,...,...
68,Adele,Adele,Rumour Has It
69,Adele,Adele,Set Fire to the Rain
70,Adele,Adele,Someone Like You
71,Adele,Adele,Take It All


In [6]:
df_songlist_select = through_path()

for i in range (len(df_songlist_select)):
    artist = df_songlist_select['Artist'][i]
    title = df_songlist_select['Title'][i]
    lyrics = print_lyrics(artist,title)
    filename = save2txt(artist,title,lyrics)
    print('Filename: '+ filename)
    #print('Lyrics:'+ '\n' + lyrics)

dir: 5 Seconds of Summer
dir: Adele
Filename: lyrics/5 Seconds of Summer_18.txt
Filename: lyrics/5 Seconds of Summer_Airplanes.txt
Filename: lyrics/5 Seconds of Summer_Amnesia.txt
Filename: lyrics/5 Seconds of Summer_Babylon.txt
Filename: lyrics/5 Seconds of Summer_Beside You.txt
Filename: lyrics/5 Seconds of Summer_Better Man.txt
Filename: lyrics/5 Seconds of Summer_Broken Home.txt
Filename: lyrics/5 Seconds of Summer_Castaway.txt
Filename: lyrics/5 Seconds of Summer_Catch Fire.txt
Filename: lyrics/5 Seconds of Summer_Close As Strangers.txt
Filename: lyrics/5 Seconds of Summer_Disconnected.txt
Filename: lyrics/5 Seconds of Summer_Don t Stop.txt
Filename: lyrics/5 Seconds of Summer_Empty Wallets.txt
Filename: lyrics/5 Seconds of Summer_End Up Here.txt
Filename: lyrics/5 Seconds of Summer_English Love Affair.txt
Filename: lyrics/5 Seconds of Summer_Everything I Didn t Say.txt
Filename: lyrics/5 Seconds of Summer_Fly Away.txt
Filename: lyrics/5 Seconds of Summer_Ghost Of You.txt
Filename

In [48]:
# 刪除歌曲名稱結尾為「_0」
def songs_remove(path):
    allFileList2 = os.listdir(path)
    S_dir_artist = []
    S_dir_artist2 = []
    S_dir_title = []
    for file in allFileList2:
        if os.path.isdir(os.path.join(path,file)):
            S_dir_artist.append(file)
            
            artistPath = path + '/' + file
            artistFileList2 = os.listdir(artistPath)
            
            for file2 in artistFileList2:
                pos = file2.find("_0")
                if os.path.isfile(artistPath + '/' + file2):
                    S_dir_title.append(file2)
                    if pos != -1:
                        print(artistPath + '/' + file2)
                        os.remove(artistPath + '/' + file2)

In [49]:
path = r'D:/Anaconda3/Scripts/7 實驗進度/SpotifyLyrics/songs'
songs_remove(path)

D:/Anaconda3/Scripts/7 實驗進度/SpotifyLyrics/songs/5 Seconds of Summer/English Love Affair_0.mp3


In [66]:
# 遍歷資料夾(D:/TuneFab Spotify Music Converter/songs)，查看需要下載的歌詞
def through_path2():
    yourPath = r'D:/TuneFab Spotify Music Converter/songs' # mp3資料夾位址
    allFileList = os.listdir(yourPath)
    dir_artist = [] # 儲存歌手的list
    dir_artist2 = [] # 儲存歌手的list(寫入dataframe用)
    dir_artist3 = [] # 儲存歌手原始名稱的list(處理資料夾名稱有多位歌手的情況)
    dir_title = [] # 儲存歌名的list

    for file in allFileList: # 遍歷songs資料夾
        if os.path.isdir(os.path.join(yourPath,file)):
            print("dir: " + file) # 列出songs資料夾底下有那些歌手
            dir_artist.append(file)

            artistPath = yourPath + '/' + file # 形式：D:/TuneFab Spotify Music Converter/songs/{歌手名稱}
            #print(artistPath)
            artistFileList = os.listdir(artistPath)
            #print(artistFileList)

            for file2 in artistFileList: # 遍歷每位歌首的資料夾
                songPath = artistPath + '/' + file2
                if os.path.isfile(songPath): # 形式：D:/TuneFab Spotify Music Converter/songs/{歌手名稱}/{歌曲名稱}
                    #print("dir2: " + file2)
                    file2 = file2.replace('.mp3','').replace("’","'") #取代附檔名
                    dir_title.append(file2)
                    
                    name = songPath.split('/')[-2] # 取songPath位址倒數第二個值
                    name_list = name.split(',') 
                    dir_artist2.append(name_list[0]) # 針對資料夾有多位歌手處理，取第一個為主要演唱者
                    
                    # 原始歌手資料夾名稱
                    originalname = artistPath.split('/')[-1]
                    dir_artist3.append(originalname)

        #print('-'*80)

    # 輸出datafrme(df_songlist)和xlsx(df_songlist.xlsx)
    # 有重複歌曲就刪除，歌曲名稱結尾是_0
    string = '_0|_1|_2'
    df_songlist2 = pd.DataFrame(list(zip(dir_artist3,dir_artist2,dir_title)),columns=['Orignal','Artist','Title'])
    df_songlist_select2 = df_songlist2.loc[~df_songlist2['Title'].str.contains(string)].reset_index(drop=True)
    df_songlist_select2.to_excel('table/df_songlist_select2.xlsx',index=0)
    return df_songlist_select2

In [67]:
df_songlist_select2 = through_path2()
df_songlist_select2

dir: 5 Seconds of Summer
dir: Adele
dir: Alan Walker,Noah Cyrus,Digital Farm Animals,Juliander
dir: Alan Walker,Sophia Somajo
dir: Alan Walker,Steve Aoki,ISÁK,Omar Noir


,Orignal,Artist,Title
0,5 Seconds of Summer,5 Seconds of Summer,18
1,5 Seconds of Summer,5 Seconds of Summer,Airplanes
2,5 Seconds of Summer,5 Seconds of Summer,Amnesia
3,5 Seconds of Summer,5 Seconds of Summer,Babylon
4,5 Seconds of Summer,5 Seconds of Summer,Beside You
...,...,...,...
69,Adele,Adele,Take It All
70,Adele,Adele,Turning Tables
71,"Alan Walker,Noah Cyrus,Digital Farm Animals,Ju...",Alan Walker,All Falls Down (feat Juliander)
72,"Alan Walker,Sophia Somajo",Alan Walker,Diamond Heart
